# DataBricks - Operational JAR Logic, applied to PAT Intermediate Table - South West

## Output Files by Provider and System for hosting on Futures

### NHS England South West Intelligence and Insights
 
The **JAR Reconciliation Files** provides a granular breakdown of hospital activity (TFC/HRG Level), based on national planning logic. This script runs the activity views and then outputs files, complete with three tabs (APC, OP, AE) for each provider and system.
 

### Scripts
 
#### Provider Focus
📝 National JAR methodology - Accident and Emergency Attendance - Provider - PAT  
📝 National JAR methodology - Admitted patient care script - Provider - PAT   
📝 National JAR methodology - Outpatient Attendance script - Provider - PAT   

### About the Scripts
The PAT and FasterSUS scripts are ran using National Planning logic, the below applies to both APC and OP scripts:  
- Acute Provider Only  
- Consultant led Specific Acute activity only  
- Treatment Function Code 360 and 812 (op) is excluded  
- Excluding Private patients
 
🚑 **Accident and Emergency Attendance script**  
*This script covers Emergency Care attendances, sourced from the National PAT Intermediate EC SUS table*  

🏥 **Admitted patient care script**  
*This script covers both elective and non-elective hospital activity, sourced from the National PAT Intermediate Admitted Patient Care SUS table*  

👨‍⚕️ **Outpatient Attendance script**  
*This script covers Outpatient attendances, sourced from the National PAT Intermediate OP SUS table*  

### Built With SQL and Python in DataBricks
 
🛢️[DataBricks](About Databricks: The data and AI company | Databricks)  
🛢️[UDAL](https://rdweb.wvd.microsoft.com) 

#### Datasets in the lakemart on DataBricks
🛢️ Pat_intermediate_OPA 
🛢️ Pat_intermediate_APC 
🛢️ Pat_intermediate_AE 


## Step 1: Load the Tables  

Before we dive into the data, we need to **load the necessary tables** into the **Hive Metastore**.  

#### Why is this needed?  
Right now, the data does not exist in the **Lakemart**, until it's been included, we use this **temporary workaround**. Thanks to the **National Elective Team**, we have access to their **ERIC workspace** to make this possible.  

#### Best Practice  
Since these tables **refresh daily**, it's always best to **drop and reload** them to ensure you're working with the most up-to-date data.  


In [0]:

--Load neccassary reference tables
drop table if exists eric.Provider_Hierarchies_DB;
create table eric.Provider_Hierarchies_DB
using parquet
location "abfss://reporting@udalstdatacuratedprod.dfs.core.windows.net/unrestricted/reference/UKHD/ODS/Provider_Hierarchies/";

drop table if exists eric.Date_Full_DB;
create table eric.Date_Full_DB
using parquet
location "abfss://unrestricted@udalstdatacuratedprod.dfs.core.windows.net/reference/Internal/Reference/Date_Full/Published/1/";

drop table if exists eric.TreatmentFunctionCodes_DB;
create table eric.TreatmentFunctionCodes_DB
using parquet
location "abfss://unrestricted@udalstdatacuratedprod.dfs.core.windows.net/aggregated/UKHF/Treatment_Function/Codes1/"
options(recursiveFileLookup=true);

--Load neccassary PAT Intermediate tables

drop table if exists eric.PAT_Intermediate_Table_APC_DB;
create table eric.PAT_Intermediate_Table_APC_DB
using parquet
location "abfss://restricted@udalstdatacuratedprod.dfs.core.windows.net/patientlevel/UDALSQLMART/PatActivity/PAT_Intermediate_Table_APC/"
options(recursiveFileLookup=true);

drop table if exists eric.PAT_Intermediate_Table_OP_DB;
create table eric.PAT_Intermediate_Table_OP_DB
using parquet
location "abfss://restricted@udalstdatacuratedprod.dfs.core.windows.net/patientlevel/UDALSQLMART/PatActivity/PAT_Intermediate_Table_OP/"
options(recursiveFileLookup=true);

drop table if exists eric.PAT_Intermediate_Table_AE_DB;
create table eric.PAT_Intermediate_Table_AE_DB
using parquet
location "abfss://restricted@udalstdatacuratedprod.dfs.core.windows.net/patientlevel/UDALSQLMART/PatActivity/PAT_Intermediate_Table_AE/"
options(recursiveFileLookup=true);
 

## Step 2: Create the Views  

There are three views using SQL Spark version of the legacy SQL view initialy created in NCDR and the replicated in UDAL, for APC, OP and AE.

In [0]:
CREATE OR REPLACE TEMPORARY VIEW PAT_APC AS
SELECT
    CASE
        WHEN susapc.Provider_Current IN ('RD3', 'RDZ') THEN 'R0D'
        WHEN susapc.Provider_Current = 'RBZ' THEN 'RH8'
        WHEN susapc.Provider_Current = 'RA3' THEN 'RA7'
        WHEN susapc.Provider_Current = 'RBA' THEN 'RH5'
        WHEN susapc.Provider_Current = 'R1G' THEN 'RA9'
        WHEN susapc.Provider_Current = 'RVJ13' THEN 'RVJ'
        WHEN susapc.Provider_Current = 'RA4' THEN 'RH5'
        ELSE susapc.Provider_Current
    END AS Provider_Code,
 
    CASE
        WHEN o.organisation_name = 'ROYAL DEVON AND EXETER NHS FOUNDATION TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TAUNTON AND SOMERSET NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'NORTHERN DEVON HEALTHCARE NHS TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name IN (
            'THE ROYAL BOURNEMOUTH AND CHRISTCHURCH HOSPITALS NHS FOUNDATION TRUST',
            'POOLE HOSPITAL NHS FOUNDATION TRUST'
        ) THEN 'UNIVERSITY HOSPITAL DORSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TORBAY AND SOUTHERN DEVON HEALTH AND CARE NHS TRUST' THEN 'TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'EMERSONS GREEN NHS TREATMENT CENTRE' THEN 'NORTH BRISTOL NHS TRUST'
        WHEN o.organisation_name = 'YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'  
        WHEN o.organisation_name = 'UNIVERSITY HOSPITALS BRISTOL NHS FOUNDATION TRUST' THEN 'UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST'
        ELSE o.organisation_name  
    END AS organisation_name,
 
    o.STP_Code,
    o.STP_Name,
    right(susapc.Dimention_3, 3) AS TFC,
    T.Treatment_Function_Title,
    LEFT(susapc.Dimention_7, 5) AS HRG,
    susapc.Pat_Commissioner_Type,
    CONCAT(YEAR(susapc.Discharge_Date), LPAD(MONTH(susapc.Discharge_Date),2,'0')) AS YearMonth,
 
    susapc.LOS_unadjusted,
 
    CASE 
        WHEN susapc.Der_Management_Type = 'EM' THEN 'NE' 
        ELSE susapc.Der_Management_Type 
    END AS Der_Management_Type,
 
    CASE 
        WHEN susapc.Dimention_5 = 'A: 0 day LOS' THEN '0 Day LOS' 
        ELSE '1+ Day LOS' 
    END AS LOS,
 
    susapc.Discharge_Date,
 
    COUNT(susapc.unadjusted) AS Total_Activty_Unadj,
    COUNT(susapc.adjusted) AS Total_Activty_Adj
 
FROM eric.PAT_Intermediate_Table_APC_DB AS susapc
 
LEFT JOIN eric.Provider_Hierarchies_DB o 
    ON susapc.Provider_Current = o.Organisation_Code  
 
LEFT JOIN eric.Date_Full_DB AS d 
    ON d.Full_Date = susapc.Discharge_Date
 
LEFT JOIN eric.TreatmentFunctionCodes_DB AS T 
    ON Right(susapc.Dimention_3, 3) = T.DD_Code
 
WHERE
    susapc.Discharge_Date >= '2024-04-01'
    AND susapc.Der_Management_Type IN ('EL', 'DC', 'EM', 'NE')
    AND susapc.Provider_Current IN (
        'RD1', 'RN3', 'RNZ', 'RA7', 'RVJ', 'REF', 'RA9', 'RH8',
        'RK9', 'RBD', 'R0D', 'RTE', 'RH5'
    )

and susapc.Dimention_4 = 'Specific Acute'

 --   AND right(susapc.Dimention_3, 3) NOT IN (
 --       '199', '223', '290', '291', '331', '344', '345', '346', '360',
 --       '424', '499', '501', '504', '560', '650', '651', '652', '653',
 --       '654', '655', '656', '657', '658', '659', '660', '661', '662',
 --       '700', '710', '711', '712', '713', '715', '720', '721', '722',
 --       '723', '724', '725', '726', '727', '730', '840', '920','NULL'
 --   )
 
    AND susapc.Pat_Commissioner_Type <> 'Private Patient'
 
GROUP BY
    susapc.Provider_Current,
    o.organisation_name,
    o.STP_Code,
    o.STP_Name,
    right(susapc.Dimention_3,3),
    T.Treatment_Function_Title,
    LEFT(susapc.Dimention_7,5),
    susapc.Pat_Commissioner_Type,
    susapc.Discharge_Date,
    susapc.LOS_unadjusted,
    susapc.Der_Management_Type,
    susapc.Dimention_5


In [0]:
CREATE OR REPLACE TEMPORARY VIEW PAT_OP AS
SELECT
    CASE
        WHEN susop.Provider_Current IN ('RD3', 'RDZ') THEN 'R0D'
        WHEN susop.Provider_Current = 'RBZ' THEN 'RH8'
        WHEN susop.Provider_Current = 'RA3' THEN 'RA7'
        WHEN susop.Provider_Current = 'RBA' THEN 'RH5'
        WHEN susop.Provider_Current = 'R1G' THEN 'RA9'
        WHEN susop.Provider_Current = 'RVJ13' THEN 'RVJ'
        WHEN susop.Provider_Current = 'RA4' THEN 'RH5'
        ELSE susop.Provider_Current
    END AS Provider_Code,
 
    CASE
        WHEN o.organisation_name = 'ROYAL DEVON AND EXETER NHS FOUNDATION TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TAUNTON AND SOMERSET NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'NORTHERN DEVON HEALTHCARE NHS TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name IN (
            'THE ROYAL BOURNEMOUTH AND CHRISTCHURCH HOSPITALS NHS FOUNDATION TRUST',
            'POOLE HOSPITAL NHS FOUNDATION TRUST'
        ) THEN 'UNIVERSITY HOSPITAL DORSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TORBAY AND SOUTHERN DEVON HEALTH AND CARE NHS TRUST' THEN 'TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'EMERSONS GREEN NHS TREATMENT CENTRE' THEN 'NORTH BRISTOL NHS TRUST'
        WHEN o.organisation_name = 'YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'  
        WHEN o.organisation_name = 'UNIVERSITY HOSPITALS BRISTOL NHS FOUNDATION TRUST' THEN 'UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST'
        ELSE o.organisation_name  
    END AS organisation_name,
 
    o.STP_Code,
    o.STP_Name,
    left(susop.Dimention_3, 3) AS TFC,
    T.Treatment_Function_Title,
    LEFT(susop.Dimention_7, 5) AS HRG,
    susop.Pat_Commissioner_Type,
    CONCAT(YEAR(susop.Attendance_Date), LPAD(MONTH(susop.Attendance_Date),2,'0')) AS YearMonth,
    
    CASE 
        WHEN susop.Dimention_1 LIKE 'Follow%' THEN 'Outpatient Follow-Up'
        WHEN susop.Dimention_1 LIKE '1st%' THEN 'Outpatient First Appointment'
        ELSE susop.Dimention_1 
    END AS High_Level_Pod,
 
    susop.Attendance_Date,
 
    COUNT(susop.unadjusted) AS Total_Activty_Unadj,
    COUNT(susop.adjusted) AS Total_Activty_Adj
 
FROM eric.PAT_Intermediate_Table_OP_DB AS susop
 
LEFT JOIN eric.Provider_Hierarchies_DB o 
    ON susop.Provider_Current = o.Organisation_Code  
 
LEFT JOIN eric.Date_Full_DB AS d 
    ON d.Full_Date = susop.Attendance_Date
 
LEFT JOIN eric.TreatmentFunctionCodes_DB AS T 
    ON left(susop.Dimention_3, 3) = T.DD_Code
 
WHERE
    susop.Attendance_Date >= '2024-04-01'

    AND susop.Provider_Current IN (
        'RD1', 'RN3', 'RNZ', 'RA7', 'RVJ', 'REF', 'RA9', 'RH8',
        'RK9', 'RBD', 'R0D', 'RTE', 'RH5'
    )

and susop.Dimention_4 = 'Consultant led: Specific Acute'

and susop.Dimention_1 <> 'Unknown Appointment Type'

 --   AND left(susapc.Dimention_3, 3) NOT IN (
 --       '199', '223', '290', '291', '331', '344', '345', '346', '360',
 --       '424', '499', '501', '504', '560', '650', '651', '652', '653',
 --       '654', '655', '656', '657', '658', '659', '660', '661', '662',
 --       '700', '710', '711', '712', '713', '715', '720', '721', '722',
 --       '723', '724', '725', '726', '727', '730', '840', '920','NULL'
 --   )
 
    AND susop.Pat_Commissioner_Type <> 'Private Patient'
 
GROUP BY
susop.Provider_Current,
o.organisation_name,
o.STP_Code,
o.STP_Name,
left(susop.Dimention_3, 3),
T.Treatment_Function_Title,
LEFT(susop.Dimention_7, 5),
susop.Pat_Commissioner_Type,
Attendance_Date,
Dimention_1,
susop.Attendance_Date;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW PAT_AE AS
SELECT
    CASE
        WHEN susae.Provider_Current IN ('RD3', 'RDZ') THEN 'R0D'
        WHEN susae.Provider_Current = 'RBZ' THEN 'RH8'
        WHEN susae.Provider_Current = 'RA3' THEN 'RA7'
        WHEN susae.Provider_Current = 'RBA' THEN 'RH5'
        WHEN susae.Provider_Current = 'R1G' THEN 'RA9'
        WHEN susae.Provider_Current = 'RVJ13' THEN 'RVJ'
        WHEN susae.Provider_Current = 'RA4' THEN 'RH5'
        ELSE susae.Provider_Current
    END AS Provider_Code,
 
    CASE
        WHEN o.organisation_name = 'ROYAL DEVON AND EXETER NHS FOUNDATION TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TAUNTON AND SOMERSET NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'NORTHERN DEVON HEALTHCARE NHS TRUST' THEN 'ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST'
        WHEN o.organisation_name IN (
            'THE ROYAL BOURNEMOUTH AND CHRISTCHURCH HOSPITALS NHS FOUNDATION TRUST',
            'POOLE HOSPITAL NHS FOUNDATION TRUST'
        ) THEN 'UNIVERSITY HOSPITAL DORSET NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'TORBAY AND SOUTHERN DEVON HEALTH AND CARE NHS TRUST' THEN 'TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST'
        WHEN o.organisation_name = 'EMERSONS GREEN NHS TREATMENT CENTRE' THEN 'NORTH BRISTOL NHS TRUST'
        WHEN o.organisation_name = 'YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST' THEN 'SOMERSET NHS FOUNDATION TRUST'  
        WHEN o.organisation_name = 'UNIVERSITY HOSPITALS BRISTOL NHS FOUNDATION TRUST' THEN 'UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST'
        ELSE o.organisation_name  
    END AS organisation_name,
 
    o.STP_Code,
    o.STP_Name,
    LEFT(susae.Dimention_7, 5) AS HRG,
    susae.Pat_Commissioner_Type,
    CONCAT(YEAR(susae.Attendance_Date), LPAD(MONTH(susae.Attendance_Date),2,'0')) AS YearMonth,
    
    susae.Dimention_1 AS Metric_ID,
 
    susae.Attendance_Date,
 
    COUNT(susae.unadjusted) AS Total_Activty_Unadj,
    COUNT(susae.adjusted) AS Total_Activty_Adj
 
FROM eric.PAT_Intermediate_Table_AE_DB AS susae
 
LEFT JOIN eric.Provider_Hierarchies_DB o 
    ON susae.Provider_Current = o.Organisation_Code  
 
LEFT JOIN eric.Date_Full_DB AS d 
    ON d.Full_Date = susae.Attendance_Date

 WHERE
    susae.Attendance_Date >= '2024-04-01'

    AND susae.Provider_Current IN (
        'RD1', 'RN3', 'RNZ', 'RA7', 'RVJ', 'REF', 'RA9', 'RH8',
        'RK9', 'RBD', 'R0D', 'RTE', 'RH5'
    )


    AND susae.Pat_Commissioner_Type <> 'Private Patient'
 
GROUP BY
susae.Provider_Current,
o.organisation_name,
o.STP_Code,
o.STP_Name,
LEFT(susae.Dimention_7, 5),
susae.Pat_Commissioner_Type,
Attendance_Date,
Dimention_1,
susae.Attendance_Date;

## Step 4: Remove old files and folders  

Before running the new files, delete the old ones.

In [0]:
%python
import shutil
import os
 
local_path = r"JAR Activity - Productivity Output Files"
 
# Check if the directory exists
if os.path.exists(local_path):
    shutil.rmtree(local_path)  # Deletes the entire directory
    print(f"Deleted: {local_path}")
else:
    print(f"Directory does not exist: {local_path}")

Deleted: JAR Activity - Productivity Output Files


## Step 4: Create the Productivity Outputs  

Using the views, generate files for each provider and system.


In [0]:
%python
 
from pyspark.sql import SparkSession
import pandas as pd
import os
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
 
# Local OneDrive Path (Change if needed)
local_path = r"JAR Activity - Productivity Output Files"
 
# Ensure directory exists
os.makedirs(local_path, exist_ok=True)
 
# Read the SQL Views
df_apc = spark.sql("SELECT * FROM PAT_APC")
df_op = spark.sql("SELECT * FROM PAT_OP")
df_ae = spark.sql("SELECT * FROM PAT_AE")
 
# Convert to Pandas
df_apc_pandas = df_apc.toPandas()
df_op_pandas = df_op.toPandas()
df_ae_pandas = df_ae.toPandas()
 
# Get the latest available date (MMM-YYYY format)
latest_date = pd.to_datetime(df_apc_pandas["Discharge_Date"]).max().strftime('%b-%Y')
 
# Function to save DataFrame to an Excel file with multiple sheets
def save_to_excel_with_tables(apc_df, op_df, ae_df, file_name):
    file_path = os.path.join(local_path, file_name)
    wb = Workbook()
    # Remove the default sheet created by Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    # Function to add DataFrame to a sheet with a table
    def add_sheet_with_table(wb, df, sheet_name):
        sheet = wb.create_sheet(title=sheet_name)
        for row in dataframe_to_rows(df, index=False, header=True):
            sheet.append(row)
        table = Table(displayName=f"{sheet_name}Table", ref=f"A1:{chr(64+df.shape[1])}{df.shape[0]+1}")
        style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False, showLastColumn=False, showRowStripes=True, showColumnStripes=True)
        table.tableStyleInfo = style
        sheet.add_table(table)
    add_sheet_with_table(wb, apc_df, "APC")
    add_sheet_with_table(wb, op_df, "OP")
    add_sheet_with_table(wb, ae_df, "AE")
    wb.save(file_path)
    print(f"Saved: {file_path}")
 
# Split and Save by Organisation Name
for org in df_apc_pandas["organisation_name"].unique():
    org_apc_df = df_apc_pandas[df_apc_pandas["organisation_name"] == org]
    org_op_df = df_op_pandas[df_op_pandas["organisation_name"] == org]
    org_ae_df = df_ae_pandas[df_ae_pandas["organisation_name"] == org]
    org_clean = org.replace(" ", "_").replace("/", "-")
    save_to_excel_with_tables(org_apc_df, org_op_df, org_ae_df, f"{org_clean}_{latest_date}.xlsx")
 
# Split and Save by STP Name
for stp in df_apc_pandas["STP_Name"].unique():
    stp_apc_df = df_apc_pandas[df_apc_pandas["STP_Name"] == stp]
    stp_op_df = df_op_pandas[df_op_pandas["STP_Name"] == stp]
    stp_ae_df = df_ae_pandas[df_ae_pandas["STP_Name"] == stp]
    stp_clean = stp.replace(" ", "_").replace("/", "-")
    save_to_excel_with_tables(stp_apc_df, stp_op_df, stp_ae_df, f"{stp_clean}_{latest_date}.xlsx")
 
print("✅ Excel files successfully saved in JAR Activity - Productivity Output Files!")
 

 

Saved: JAR Activity - Productivity Output Files/UNIVERSITY_HOSPITALS_DORSET_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/UNIVERSITY_HOSPITALS_PLYMOUTH_NHS_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/NORTH_BRISTOL_NHS_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/ROYAL_CORNWALL_HOSPITALS_NHS_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/ROYAL_UNITED_HOSPITALS_BATH_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/DORSET_COUNTY_HOSPITAL_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/GLOUCESTERSHIRE_HOSPITALS_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/GREAT_WESTERN_HOSPITALS_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/SOMERSET_NHS_FOUNDATION_TRUST_Jan-2025.xlsx
Saved: JAR Activity - Productivity Output Files/ROYAL_DEVON_UNIVERSITY_HEALTHCA

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

## Step 5: Create Final Productivity Output File 

Using the views, generate SW totals file to QA highlevel to JAR

In [0]:
%python
from pyspark.sql import SparkSession
import pandas as pd
import os
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
 
# Local OneDrive Path (Change if needed)
local_path = r"JAR Activity - Productivity Output Files"
 
# Ensure directory exists
os.makedirs(local_path, exist_ok=True)
 
# Read the SQL Views
df_apc = spark.sql("SELECT * FROM PAT_APC")
df_op = spark.sql("SELECT * FROM PAT_OP")
df_ae = spark.sql("SELECT * FROM PAT_AE")
 
# Convert to Pandas
df_apc_pandas = df_apc.toPandas()
df_op_pandas = df_op.toPandas()
df_ae_pandas = df_ae.toPandas()
 
# Drop unnecessary fields
drop_columns = ["HRG", "TFC", "organisation_name", "STP_Name", "STP_Code"]
df_apc_pandas = df_apc_pandas.drop(columns=[col for col in drop_columns if col in df_apc_pandas.columns], errors='ignore')
df_op_pandas = df_op_pandas.drop(columns=[col for col in drop_columns if col in df_op_pandas.columns], errors='ignore')
df_ae_pandas = df_ae_pandas.drop(columns=[col for col in drop_columns if col in df_ae_pandas.columns], errors='ignore')
 
# Aggregate total activity columns
def aggregate_activity(df, group_by_cols):
    sum_cols = ["Total_Activty_Unadj", "Total_Activty_Adj"]
    return df.groupby(group_by_cols, as_index=False)[sum_cols].sum()
 
# Define grouping columns including provider codes
apc_group_by = ["Der_Management_Type", "Provider_Code"]
ae_group_by = ["Metric_ID", "Provider_Code"]
op_group_by = ["High_Level_Pod", "Provider_Code"]
 
# Aggregate data
df_apc_pandas = aggregate_activity(df_apc_pandas, apc_group_by)
df_op_pandas = aggregate_activity(df_op_pandas, op_group_by)
df_ae_pandas = aggregate_activity(df_ae_pandas, ae_group_by)
 
# Get the latest available date (MMM-YYYY format)
latest_date = pd.Timestamp.today().strftime('%b-%Y')
 
# Function to save DataFrame to an Excel file with multiple sheets
def save_to_excel_with_tables(apc_df, op_df, ae_df, file_name):
    file_path = os.path.join(local_path, file_name)
    wb = Workbook()
    # Remove the default sheet created by Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    # Function to add DataFrame to a sheet with a table
    def add_sheet_with_table(wb, df, sheet_name):
        sheet = wb.create_sheet(title=sheet_name)
        for row in dataframe_to_rows(df, index=False, header=True):
            sheet.append(row)
        table = Table(displayName=f"{sheet_name}Table", ref=f"A1:{chr(64+df.shape[1])}{df.shape[0]+1}")
        style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False, showLastColumn=False, showRowStripes=True, showColumnStripes=True)
        table.tableStyleInfo = style
        sheet.add_table(table)
    add_sheet_with_table(wb, apc_df, "APC")
    add_sheet_with_table(wb, op_df, "OP")
    add_sheet_with_table(wb, ae_df, "AE")
    wb.save(file_path)
    print(f"Saved: {file_path}")
 
# Save total data in a single file
save_to_excel_with_tables(df_apc_pandas, df_op_pandas, df_ae_pandas, f"Total_{latest_date}.xlsx")
 
print("✅ Single Excel file with reduced size successfully saved in JAR Activity - Productivity Output Files!")
 

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

### Contact
 
To find out more about the South West Intelligence and Insights Team visit our [South West Intelligence and Insights Team Futures Page](https://future.nhs.uk/SouthWestAnalytics)) or get in touch at [england.southwestanalytics@nhs.net](mailto:england.southwestanalytics@nhs.net). Alternatively, Please feel free to reach out to me directly:
 
📧 Email: [Destiny.Bradley@nhs.net](mailto:Destiny.Bradley@nhs.net)  
💬 Teams: [Join my Teams](https://teams.microsoft.com/l/chat/0/0?users=<destiny.bradley@nhs.net)
 
### Acknowledgements
Thanks to Bernardo Detanico for his ongoing support in applying National Logic and Miles Filton for his support with getting started on Databricks